# 05. Inference & Submission Generation

## 개요
학습된 모델을 사용하여 테스트 데이터에 대한 추론을 수행하고 제출 파일을 생성하는 노트북입니다.

## 주요 기능
- 저장된 모델 자동 로드 (feature 정보 포함)
- 테스트 데이터 배치별 추론
- 메모리 효율적 처리
- 베이스라인 호환 제출 파일 생성

## 실행 방법
1. **학습 후 바로 실행**: 04번 노트북 실행 직후 → 변수들이 메모리에 있음
2. **별도 실행**: 01→02→03→05 순서로 실행 → 저장된 모델 자동 로드

**주의**: 이 노트북을 실행하기 전에 **01**, **02**, **03** 노트북을 먼저 실행해주세요!


## Model Loading


In [ ]:
# 모델 및 feature 정보 확인/로드
print("="*80)
print("STARTING ENHANCED INFERENCE WITH TRAINED MODEL")
print("="*80)

# 04번 노트북에서 이어받은 변수들이 있는지 확인
if 'training_completed' in locals() and 'trained_model' in locals() and 'trained_feature_cols' in locals():
    print("Using variables from training session:")
    print(f"- training_completed: {training_completed}")
    print(f"- trained_model: Available")
    print(f"- trained_feature_cols: {len(trained_feature_cols)} features")
    
    model = trained_model
    inference_feature_cols = trained_feature_cols
    print("Model and features loaded from training session!")
    
else:
    print("Loading model from saved checkpoint...")
    
    # 최종 모델 경로
    final_model_path = os.path.join(CFG['MODELS_PATH'], f"{CFG['MODEL_NAME']}_enhanced_final.pth")
    
    if not os.path.exists(final_model_path):
        print(f"Final model not found: {final_model_path}")
        print("Trying to find latest checkpoint...")
        
        # 체크포인트 파일 찾기
        checkpoint_files = glob.glob(os.path.join(CFG['MODELS_PATH'], f"{CFG['MODEL_NAME']}_enhanced_checkpoint_*.pth"))
        if checkpoint_files:
            final_model_path = max(checkpoint_files)  # 가장 최근 체크포인트
            print(f"Using latest checkpoint: {os.path.basename(final_model_path)}")
        else:
            raise FileNotFoundError("No trained model found! Please run 04_training.ipynb first.")
    
    # 모델 로드
    model, inference_feature_cols = load_model_for_inference(final_model_path, device=device)
    
    if inference_feature_cols is None:
        print("Feature columns not found in checkpoint. Extracting from split data...")
        # Split 파일에서 feature 정보 추출
        split_files = glob.glob(os.path.join(CFG['SPLIT_DATA_PATH'], "part_*.parquet"))
        if split_files:
            temp_df = pd.read_parquet(split_files[0], nrows=1000)
            inference_feature_cols = get_feature_columns(temp_df)
            del temp_df
            clear_memory()
        else:
            raise FileNotFoundError("No split files found for feature extraction!")
    
    print(f"Model loaded successfully!")
    print(f"Features available: {len(inference_feature_cols)}")

print(f"Final feature count: {len(inference_feature_cols)}")
print(f"Memory after model loading: {get_memory_usage()}")
model.eval()  # 추론 모드로 설정


## Test Data Loading


In [ ]:
# 테스트 데이터 로드
print("="*60)
print("Loading Test Data")
print("="*60)

test_df = pd.read_parquet("../../data/raw/test.parquet", engine="pyarrow")
print(f"Test data shape: {test_df.shape}")

# ID 컬럼 따로 보관 (제출용)
test_ids = test_df['ID'].copy()

# ID 컬럼 제거 (feature에 포함되지 않음)
test_df = test_df.drop(columns=['ID'])

print(f"Test data shape after removing ID: {test_df.shape}")
print(f"Memory after test data loading: {get_memory_usage()}")

# 데이터셋 및 DataLoader 생성
seq_col = "seq"
test_dataset = ClickDataset(test_df, inference_feature_cols, seq_col, has_target=False)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, collate_fn=collate_fn_infer)

print(f"Test dataset size: {len(test_dataset):,}")
print(f"Number of batches: {len(test_loader)}")
print(f"Batch size: {CFG['BATCH_SIZE']:,}")


## Inference Function


In [ ]:
def perform_inference(model, test_loader, device):
    """
    메모리 효율적 배치별 추론 수행
    """
    print("="*60)
    print("Starting Model Inference")
    print("="*60)
    
    predictions = []
    start_time = datetime.now()
    batch_times = []
    
    with torch.no_grad():
        inference_progress = tqdm(test_loader, desc="Inference Progress")
        for batch_idx, (xs, seqs, lens) in enumerate(inference_progress):
            batch_start = datetime.now()
            
            xs, seqs, lens = xs.to(device), seqs.to(device), lens.to(device)
            
            # 모델 예측
            logits = model(xs, seqs, lens)
            probs = torch.sigmoid(logits)
            
            predictions.append(probs.cpu())
            
            # 배치 처리 시간 기록
            batch_time = (datetime.now() - batch_start).total_seconds()
            batch_times.append(batch_time)
            
            # Progress bar 업데이트
            inference_progress.set_postfix({
                'batch': f"{batch_idx+1}/{len(test_loader)}",
                'time': f"{batch_time:.3f}s"
            })
            
            # 주기적 메모리 정리 (100배치마다)
            if (batch_idx + 1) % 100 == 0:
                clear_memory()
                avg_batch_time = np.mean(batch_times[-100:])
                print(f"Processed {batch_idx+1} batches | Avg batch time: {avg_batch_time:.3f}s")
    
    # 예측 결과 합치기
    final_predictions = torch.cat(predictions).numpy()
    
    # 통계 정보
    end_time = datetime.now()
    total_time = (end_time - start_time).total_seconds()
    avg_batch_time = np.mean(batch_times)
    
    print(f"\nInference completed successfully!")
    print(f"Total inference time: {total_time:.2f} seconds")
    print(f"Average batch time: {avg_batch_time:.3f} seconds")
    print(f"Predictions shape: ({final_predictions.shape[0]:,},)")
    
    # 예측 통계
    print(f"Prediction statistics:")
    print(f"   Min: {final_predictions.min():.6f}")
    print(f"   Max: {final_predictions.max():.6f}")
    print(f"   Mean: {final_predictions.mean():.6f}")
    print(f"   Std: {final_predictions.std():.6f}")
    print(f"   Median: {np.median(final_predictions):.6f}")
    
    # 예측 분포 확인
    bins = np.arange(0, 1.1, 0.1)
    hist, _ = np.histogram(final_predictions, bins=bins)
    print(f"Prediction distribution:")
    for i in range(len(bins)-1):
        pct = hist[i] / len(final_predictions) * 100
        print(f"   {bins[i]:.1f}-{bins[i+1]:.1f}: {hist[i]:,} ({pct:.1f}%)")
    
    return final_predictions

# 추론 실행
print("Starting inference...")
test_predictions = perform_inference(model, test_loader, device)
print(f"Memory after inference: {get_memory_usage()}")


## Submission File Generation


In [ ]:
# 제출 파일 생성 (베이스라인 호환 양식)
print("="*60)
print("Creating Submission File")
print("="*60)

# 기존 sample_submission.csv 파일 읽기
sample_submission = pd.read_csv('../../data/raw/sample_submission.csv')
print(f"Sample submission shape: {sample_submission.shape}")

# 예측 결과로 clicked 컬럼 업데이트
submission_df = sample_submission.copy()
submission_df['clicked'] = test_predictions

# ID 순서 확인 (안전성 체크)
if not submission_df['ID'].equals(test_ids):
    print("WARNING: ID order mismatch! Re-aligning...")
    # ID를 기준으로 병합
    test_results = pd.DataFrame({'ID': test_ids, 'clicked': test_predictions})
    submission_df = sample_submission[['ID']].merge(test_results, on='ID', how='left')

print(f"Submission DataFrame shape: {submission_df.shape}")
print(f"Submission predictions stats:")
print(f"  Min: {test_predictions.min():.6f}")
print(f"  Max: {test_predictions.max():.6f}")
print(f"  Mean: {test_predictions.mean():.6f}")
print(f"  Std: {test_predictions.std():.6f}")

# outputs 폴더 확인 및 생성
output_dir = '../../outputs'
os.makedirs(output_dir, exist_ok=True)

# 기존 제출 파일 확인하여 번호 결정 (베이스라인과 동일한 방식)
existing_files = [f for f in os.listdir(output_dir) if f.startswith('submission_') and f.endswith('.csv')]

if len(existing_files) == 0:
    next_num = 1
    print("No existing submission files found. Starting with submission_1.csv")
else:
    nums = [int(f.split('_')[1].split('.')[0]) for f in existing_files]
    next_num = max(nums) + 1
    print(f"Found {len(existing_files)} existing submission files. Next: submission_{next_num}.csv")

# 새로운 파일명으로 저장 (베이스라인과 동일한 양식)
output_path = os.path.join(output_dir, f'submission_{next_num}.csv')

submission_df.to_csv(output_path, index=False)

print(f"Submission file saved: {output_path}")
print(f"Submission shape: {submission_df.shape}")
print(f"File: submission_{next_num}.csv")

# 최종 메모리 정리
clear_memory()
print(f"Final memory usage: {get_memory_usage()}")

print(f"\n{'='*60}")
print("INFERENCE AND SUBMISSION COMPLETED SUCCESSFULLY!")
print(f"{'='*60}")
print(f"Submission file: submission_{next_num}.csv")
print(f"Location: {output_path}")
print(f"Ready for submission!")
